# Voorbeeld video: Douche Environment
Dit voorbeeld mee gedaan met de video over je eigen environment bouwen, om te leren hoe ik dat zo voor ons containergrid kan doen. Vanuit dit voorbeeld om gaan bouwen naar ons probleem en vervolgens de reward functie gaan uitbouwen.

Halverwege in t kopje "Environment douche namaken" heb ik een lijstje met dingen die ik wil aanpassen, en hoe, in container situatie opgeschreven.

### Imports

In [1]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

import numpy as np
import random
import os

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

### Oefenen verschillende spaces
GymAI heeft verschillende spaces waar je mee kan werken.

In [9]:
# integer tussen 0 en 3 (max is 2)
Discrete(3).sample()

0

In [10]:
# array 3 lijsten van 3, float tussen 0 en 1
Box(0,1,shape=(3,3)).sample()

array([[0.5299681 , 0.156079  , 0.8387429 ],
       [0.7959655 , 0.94332254, 0.96360224],
       [0.25915903, 0.66592073, 0.08170765]], dtype=float32)

In [11]:
# geen floats maar integers
Box(0,255,shape=(3,3), dtype=int).sample()

array([[167, 124,  95],
       [ 26, 239,  97],
       [255,   4,  54]])

In [31]:
Box(low=1, high=3, shape=(1, 3), dtype=np.int64).sample().tolist()

[[2, 1, 1]]

In [12]:
# meerdere spaces in tuple vorm
Tuple((Discrete(2), Box(0,100, shape=(1,)))).sample()

(0, array([54.83357], dtype=float32))

In [13]:
# meerdere spaces gelinkt aan keys in dictionary
Dict({'height':Discrete(2), "speed":Box(0,100, shape=(1,))}).sample()

OrderedDict([('height', 1), ('speed', array([14.587363], dtype=float32))])

In [14]:
# array lijst van 4 met 0en en 1en
MultiBinary(4).sample()

array([1, 0, 0, 1], dtype=int8)

In [29]:
# array lijst met op index 0: 0 t/m 4, index 1: 0 t/m 1, index 2: 0 t/m 1. integers
MultiDiscrete([5,2,2]).sample()

array([3, 0, 1])

### Environment douche namaken

In [59]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length
        self.shower_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self, mode = 'human'):
        # Implement viz
        display(self.state)
        #pass # hier kan bijv een display(self.state) als je die wil zien
    
    def reset(self):
        # Reset shower temperature
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        # Reset shower time
        self.shower_length = 60 
        return self.state

Als zometeen containerprobleem grid:
- action_space in Multidiscrete als coordinaat [x,y] in grid
- observation_space als box met array van 3 lijsten van 3 (matrix)
- shower_length zal dan aantal zetten zijn, dus 9, want 9 containers plaatsen voordat een game over is.
- reward functie zelf schrijven voor hoe goed plaatsing van container is + of het feasible is
- naast self.state moet het model ook een lijst hebben van containers die nog in t grid geplaatst moeten worden

In [60]:
env=ShowerEnv()
env.observation_space.sample()

array([1.9839275], dtype=float32)

In [61]:
env.reset()

array([38.])

In [62]:
from stable_baselines3.common.env_checker import check_env
check_env(env, warn=True)

### Env testen

In [63]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

array([40.])

array([41.])

array([42.])

array([42.])

array([43.])

...

array([49.])

array([49.])

array([49.])

array([49.])

Episode:5 Score:-58


In [64]:
env.close()

### Trainen

In [65]:
#log_path = os.path.join('Training', 'Logs')
model = PPO("MlpPolicy", env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [66]:
model.learn(total_timesteps=400000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -25.1    |
| time/              |          |
|    fps             | 2161     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 60            |
|    ep_rew_mean          | -29.2         |
| time/                   |               |
|    fps                  | 1642          |
|    iterations           | 2             |
|    time_elapsed         | 2             |
|    total_timesteps      | 4096          |
| train/                  |               |
|    approx_kl            | 0.00068454974 |
|    clip_fraction        | 0.00747       |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.09         |
|    explained_variance   | 0.000327      |


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | -21.1        |
| time/                   |              |
|    fps                  | 1329         |
|    iterations           | 11           |
|    time_elapsed         | 16           |
|    total_timesteps      | 22528        |
| train/                  |              |
|    approx_kl            | 0.0059396382 |
|    clip_fraction        | 0.0776       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.07        |
|    explained_variance   | -0.00836     |
|    learning_rate        | 0.0003       |
|    loss                 | 35.1         |
|    n_updates            | 100          |
|    policy_gradient_loss | -0.00581     |
|    value_loss           | 72.5         |
------------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_m

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -1.14       |
| time/                   |             |
|    fps                  | 1283        |
|    iterations           | 21          |
|    time_elapsed         | 33          |
|    total_timesteps      | 43008       |
| train/                  |             |
|    approx_kl            | 0.009231163 |
|    clip_fraction        | 0.11        |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.05       |
|    explained_variance   | 0.000232    |
|    learning_rate        | 0.0003      |
|    loss                 | 27.9        |
|    n_updates            | 200         |
|    policy_gradient_loss | -0.00603    |
|    value_loss           | 51.7        |
-----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 60      

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 40.9        |
| time/                   |             |
|    fps                  | 1277        |
|    iterations           | 31          |
|    time_elapsed         | 49          |
|    total_timesteps      | 63488       |
| train/                  |             |
|    approx_kl            | 0.009325253 |
|    clip_fraction        | 0.19        |
|    clip_range           | 0.2         |
|    entropy_loss         | -1          |
|    explained_variance   | -3.97e-05   |
|    learning_rate        | 0.0003      |
|    loss                 | 30.8        |
|    n_updates            | 300         |
|    policy_gradient_loss | -0.00297    |
|    value_loss           | 58.5        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60    

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | 33           |
| time/                   |              |
|    fps                  | 1259         |
|    iterations           | 41           |
|    time_elapsed         | 66           |
|    total_timesteps      | 83968        |
| train/                  |              |
|    approx_kl            | 0.0123608485 |
|    clip_fraction        | 0.178        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.962       |
|    explained_variance   | 1.36e-05     |
|    learning_rate        | 0.0003       |
|    loss                 | 45.5         |
|    n_updates            | 400          |
|    policy_gradient_loss | -0.00232     |
|    value_loss           | 77.8         |
------------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_m

...

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 60         |
|    ep_rew_mean          | 30.9       |
| time/                   |            |
|    fps                  | 1258       |
|    iterations           | 151        |
|    time_elapsed         | 245        |
|    total_timesteps      | 309248     |
| train/                  |            |
|    approx_kl            | 0.25443366 |
|    clip_fraction        | 0.111      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.24      |
|    explained_variance   | 0.00087    |
|    learning_rate        | 0.0003     |
|    loss                 | 74         |
|    n_updates            | 1500       |
|    policy_gradient_loss | -0.00235   |
|    value_loss           | 176        |
----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 60         |
|    ep_rew_mean

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 51.7        |
| time/                   |             |
|    fps                  | 1257        |
|    iterations           | 161         |
|    time_elapsed         | 262         |
|    total_timesteps      | 329728      |
| train/                  |             |
|    approx_kl            | 0.020514471 |
|    clip_fraction        | 0.0658      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.225      |
|    explained_variance   | -0.000266   |
|    learning_rate        | 0.0003      |
|    loss                 | 52.1        |
|    n_updates            | 1600        |
|    policy_gradient_loss | 0.0107      |
|    value_loss           | 102         |
-----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 60      

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 46.1        |
| time/                   |             |
|    fps                  | 1256        |
|    iterations           | 171         |
|    time_elapsed         | 278         |
|    total_timesteps      | 350208      |
| train/                  |             |
|    approx_kl            | 0.051360063 |
|    clip_fraction        | 0.124       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.297      |
|    explained_variance   | 0.000262    |
|    learning_rate        | 0.0003      |
|    loss                 | 66.7        |
|    n_updates            | 1700        |
|    policy_gradient_loss | -0.00209    |
|    value_loss           | 135         |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60    

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 54.8        |
| time/                   |             |
|    fps                  | 1253        |
|    iterations           | 181         |
|    time_elapsed         | 295         |
|    total_timesteps      | 370688      |
| train/                  |             |
|    approx_kl            | 0.017277459 |
|    clip_fraction        | 0.0641      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.255      |
|    explained_variance   | 2.62e-06    |
|    learning_rate        | 0.0003      |
|    loss                 | 75.2        |
|    n_updates            | 1800        |
|    policy_gradient_loss | 0.00589     |
|    value_loss           | 113         |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60    

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 52.4        |
| time/                   |             |
|    fps                  | 1252        |
|    iterations           | 191         |
|    time_elapsed         | 312         |
|    total_timesteps      | 391168      |
| train/                  |             |
|    approx_kl            | 0.027989103 |
|    clip_fraction        | 0.107       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.361      |
|    explained_variance   | -7.16e-05   |
|    learning_rate        | 0.0003      |
|    loss                 | 56          |
|    n_updates            | 1900        |
|    policy_gradient_loss | 0.00756     |
|    value_loss           | 119         |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60    

### Model gebruiken/testen

In [67]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

array([38.])

array([38.])

array([38.])

array([38.])

array([38.])

...

array([39.])

array([39.])

array([39.])

array([36.])

(47.6, 35.87533971964586)